In [6]:
import pandas as pd
import numpy as np
from pathlib import Path

# ============================================================
# 경로 설정
BASE_DIR = Path(r"Z:\WERL\Team\shlee\Sobol\8_analysis_results\Observations")
INPUT_FILE = BASE_DIR / "02_Annual_Stats_WaterYear.csv"
OUT_DIR = BASE_DIR / "Gumbel"
OUT_DIR.mkdir(exist_ok=True)

# AMS 컬럼명 (⚠ 반드시 확인!)
FLOW_COL = "annual_max_cms"   # 예: "max", "FLOW_OUTcms_max" 등

# 분석 설정
RETURN_PERIODS = [2, 5, 10, 25, 50, 100]
N_BOOT = 5000
CI = 0.95
RANDOM_SEED = 42
# ============================================================

# === 데이터 로드 ===
df = pd.read_csv(INPUT_FILE)
ams = df[FLOW_COL].dropna().values
n = len(ams)

# === Gumbel (Method of Moments) ===
def fit_gumbel_mom(x):
    mean = np.mean(x)
    std = np.std(x, ddof=1)
    beta = std * np.sqrt(6) / np.pi
    mu = mean - 0.5772 * beta
    return mu, beta

def gumbel_quantile(mu, beta, T):
    return mu - beta * np.log(-np.log(1 - 1 / T))

# === 기준 Gumbel ===
mu_hat, beta_hat = fit_gumbel_mom(ams)

Q_mean = {
    T: gumbel_quantile(mu_hat, beta_hat, T)
    for T in RETURN_PERIODS
}

# === Bootstrap ===
rng = np.random.default_rng(RANDOM_SEED)
Q_boot = {T: [] for T in RETURN_PERIODS}

for _ in range(N_BOOT):
    sample = rng.choice(ams, size=n, replace=True)
    mu_b, beta_b = fit_gumbel_mom(sample)
    for T in RETURN_PERIODS:
        Q_boot[T].append(gumbel_quantile(mu_b, beta_b, T))

# === Confidence Interval ===
alpha = (1 - CI) / 2
records = []

for T in RETURN_PERIODS:
    q = np.array(Q_boot[T])
    records.append({
        "ReturnPeriod_yr": T,
        "Q_mean": Q_mean[T],
        "Q_DCL": np.quantile(q, alpha),
        "Q_UCL": np.quantile(q, 1 - alpha)
    })

out_df = pd.DataFrame(records)

# === 결과 저장 ===
out_csv = OUT_DIR / "Gumbel_ReturnLevels.csv"
out_df.to_csv(out_csv, index=False)

# === 메타데이터 저장 (논문 재현성용) ===
meta_txt = OUT_DIR / "Gumbel_Metadata.txt"
with open(meta_txt, "w", encoding="utf-8") as f:
    f.write("Gumbel Extreme Value Analysis (Observed)\n")
    f.write("====================================\n")
    f.write(f"Input file: {INPUT_FILE}\n")
    f.write(f"AMS column: {FLOW_COL}\n")
    f.write(f"Number of years: {n}\n")
    f.write("Return periods: " + ", ".join(map(str, RETURN_PERIODS)) + "\n")
    f.write(f"Bootstrap samples: {N_BOOT}\n")
    f.write(f"Confidence level: {CI}\n")
    f.write(f"Fitting method: Method of Moments\n")
    f.write(f"Random seed: {RANDOM_SEED}\n")
    f.write(f"mu = {mu_hat:.4f}, beta = {beta_hat:.4f}\n")

print("✔ Gumbel analysis completed")
print(f"✔ Results saved to: {out_csv}")


✔ Gumbel analysis completed
✔ Results saved to: Z:\WERL\Team\shlee\Sobol\8_analysis_results\Observations\Gumbel\Gumbel_ReturnLevels.csv
